In [ ]:
import os
import cv2
import time
import numpy as np
import scipy.fftpack as fftpack
from scipy import signal

In [ ]:
freq_min = 1
freq_max = 1.8
faceCascade = cv2.CascadeClassifier("haarcascades/haarcascade_frontalface_alt0.xml")

In [ ]:
def build_laplacian_pyramid(img, levels):
    gaussian_pyramid = cv2.pyrDown(img)

    upsampled = cv2.pyrUp(gaussian_pyramid)
    (height, width, depth) = upsampled.shape
    gaussian_pyramid = cv2.resize(gaussian_pyramid, (height, width))

    return cv2.subtract(gaussian_pyramid, upsampled)

In [ ]:
def read_video(path):
    cap = cv2.VideoCapture(path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    video_frames = []
    check = True

    while cap.isOpened():
        ret, img = cap.read()
        if not ret:
            break

        if check:
            face_detector = faceCascade.detectMultiScale(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY), 1.3, 5)
            if len(face_detector) > 0:
                print(face_detector)
                (x, y, w, h) = face_detector[0]
                check = False
        
        if not check:
            img = cv2.resize(img[y:y + h, x:x + w], (500, 500)) * (1. / 255)
            video_frames.append(build_laplacian_pyramid(img, 3))
    
    cap.release()
    
    return np.array(video_frames), fps

In [ ]:
def fft_filter(video, freq_min, freq_max, fps):
    fft = fftpack.fft(video, axis=0)
    frequencies = fftpack.fftfreq(video.shape[0], d=1.0 / fps)
    bound_low = (np.abs(frequencies - freq_min)).argmin()
    bound_high = (np.abs(frequencies - freq_max)).argmin()
    fft[:bound_low] = 0
    fft[bound_high:-bound_high] = 0
    fft[-bound_low:] = 0
    iff = fftpack.ifft(fft, axis=0)

    return fft, frequencies

In [ ]:
def find_heart_rate(fft, freqs, freq_min, freq_max):
    fft_maximums = []

    for i in range(fft.shape[0]):
        if freq_min <= freqs[i] <= freq_max:
            fftMap = abs(fft[i])
            fft_maximums.append(fftMap.max())
        else:
            fft_maximums.append(0)

    peaks, properties = signal.find_peaks(fft_maximums)
    max_peak = -1
    max_freq = 0

    for peak in peaks:
        if fft_maximums[peak] > max_freq:
            max_freq = fft_maximums[peak]
            max_peak = peak

    return freqs[max_peak] * 60

In [ ]:
path = 'videos/face.mp4'
video, fps = read_video(path)

In [ ]:
fft, frequencies = fft_filter(video, freq_min, freq_max, fps)

In [ ]:
heart_rate = find_heart_rate(fft, frequencies, freq_min, freq_max)
print("Heart rate: ", heart_rate, "bpm")